### InceptionV3 transfer learning

#### Import necessary libraries

In [1]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint 
from keras.applications.resnet50 import ResNet50


from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

import datetime


c:\program files (x86)\microsoft visual studio\shared\anaconda3_64\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Import Dataset

In [2]:
start_time = datetime.datetime.now()

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    oct_files = np.array(data['filenames'])
    oct_targets = np_utils.to_categorical(np.array(data['target']), 4)
    return oct_files, oct_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('OCT2017-RESIZED-V1/train')
valid_files, valid_targets = load_dataset('OCT2017-RESIZED-V1/valid')
test_files, test_targets = load_dataset('OCT2017-RESIZED-V1/test')

# load list of oct names
oct_names = [item[20:-1] for item in sorted(glob("OCT2017-RESIZED-V1/train/*/"))]

# print statistics about the dataset
print('There are %d total oct categories.' % len(oct_names))
print('There are %s total oct images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training oct images.' % len(train_files))
print('There are %d validation oct images.' % len(valid_files))
print('There are %d test oct images.'% len(test_files))

print('Ellapsed: ' + str(datetime.datetime.now() - start_time))

There are 4 total oct categories.
There are 7020 total oct images.

There are 5082 training oct images.
There are 938 validation oct images.
There are 1000 test oct images.
Ellapsed: 0:01:26.594943


#### Pre-process the Data

In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
start_time = datetime.datetime.now()

from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

print('Ellapsed: ' + str(datetime.datetime.now() - start_time))

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:24<00:00, 40.25it/s]


Ellapsed: 0:01:06.932360


---
<a id='step5'></a>
### Utilize InceptionV3 pretrained model to predict OCT diagnosis


- [InceptionV3](https://keras.io/applications/#inceptionv3) pretrained on imagenet


In [5]:
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.InceptionV3.hdf5', 
                               verbose=1, save_best_only=True)

#### Instantiate InceptionV3 model pretrained on imagenet

##### InceptionV3 parameters 

- include_top set to false to fine-tune the last layers to make predictions on the 4 classification categories

##### Create architecture and train model

- Freeze the first 200 layers to use pretrain weights 
- Enable training on the rest 110 layers. 
- Compile the model using the following parameters:
    - optimizer = rmsprop
    - loss = categorical_crossentropy
    
- Train the model 

- Repeat process
    - Freeze the first 200 layers to use pretrain weights 
    - Enable training on the rest 110 layers. 
    - Compile the model using the following parameters:
        - optimizer = Stochastic Gradient Descent with learning rate 0.0001 and momentum=0.9
        - loss= categorical_crossentropy
- Train model

- epochs was set to 5
    


In [6]:
start_time = datetime.datetime.now()

#InceptionV
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)

# Create the layer to make predictions on the 4 different classes using a softmax activation function.
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

epochs = 5


# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers[:200]:
   layer.trainable = False
for layer in base_model.layers[200:]:
   layer.trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name, layer.trainable)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 200 layers and unfreeze the rest:
for layer in model.layers[:200]:
   layer.trainable = False
for layer in model.layers[200:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=2)


print('Ellapsed: ' + str(datetime.datetime.now() - start_time))

Train on 5082 samples, validate on 938 samples
Epoch 1/5
 - 157s - loss: 1.1069 - val_loss: 4.8554

Epoch 00001: val_loss improved from inf to 4.85537, saving model to saved_models/weights.best.InceptionV3.hdf5
Epoch 2/5
 - 129s - loss: 0.2916 - val_loss: 4.7865

Epoch 00002: val_loss improved from 4.85537 to 4.78649, saving model to saved_models/weights.best.InceptionV3.hdf5
Epoch 3/5
 - 129s - loss: 0.1940 - val_loss: 3.8102

Epoch 00003: val_loss improved from 4.78649 to 3.81024, saving model to saved_models/weights.best.InceptionV3.hdf5
Epoch 4/5
 - 129s - loss: 0.1446 - val_loss: 6.2039

Epoch 00004: val_loss did not improve from 3.81024
Epoch 5/5
 - 129s - loss: 0.3720 - val_loss: 3.5182

Epoch 00005: val_loss improved from 3.81024 to 3.51815, saving model to saved_models/weights.best.InceptionV3.hdf5
0 input_1 False
1 conv2d_1 False
2 batch_normalization_1 False
3 activation_1 False
4 conv2d_2 False
5 batch_normalization_2 False
6 activation_2 False
7 conv2d_3 False
8 batch_norm

#### Load Model weights

In [7]:
model.load_weights('saved_models/weights.best.InceptionV3.hdf5')

#### Test model accuracy

In [8]:
# get index of predicted dog breed for each image in test set
diagnosis_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(diagnosis_predictions)==np.argmax(test_targets, axis=1))/len(diagnosis_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 81.5000%
